# Use a transcript of recorded Discovery workshop to help write sections of the discovery readout document

### You need to have `rapid7_dw_transcript.txt` in this folder, which is not committed to version control.

In [ ]:
%load_ext jupyter_black

In [ ]:
import json
import logging

import boto3
from botocore.exceptions import ClientError

logger = logging.getLogger(__name__)

### Install python bedrock SDK from reinvent (downloaded as a zip file) [[link](https://docs.aws.amazon.com/bedrock/latest/userguide/batch-inference-example.html)][[link](https://docs.aws.amazon.com/bedrock/latest/userguide/api-setup.html#api-sdk)]

In [ ]:
!mkdir -p tmp-boto-install
!unzip -qq /home/ubuntu/bedrock-python-sdk-reinvent.zip -d tmp-boto-install
!pip install tmp-boto-install/botocore-1.32.4-py3-none-any.whl --quiet
!pip install tmp-boto-install/boto3-1.29.4-py3-none-any.whl --quiet
!rm -rf tmp-boto-install

In [ ]:
# Test single prompt inference, to make sure connections work
def invoke_claude_3_with_text(prompt):
    """
    Invokes Anthropic Claude 3 Sonnet to run an inference using the input
    provided in the request body.

    :param prompt: The prompt that you want Claude 3 to complete.
    :return: Inference response from the model.
    """

    # Initialize the Amazon Bedrock runtime client
    client = boto3.client(service_name="bedrock-runtime", region_name="us-east-1")

    # Invoke Claude 3 with the text prompt
    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

    try:
        response = client.invoke_model(
            modelId=model_id,
            body=json.dumps(
                {
                    "anthropic_version": "bedrock-2023-05-31",
                    "max_tokens": 5000,
                    "messages": [
                        {
                            "role": "user",
                            "content": [{"type": "text", "text": prompt}],
                        }
                    ],
                    "temperature": 1,
                    "top_p": 0.999,
                    "top_k": 250,
                }
            ),
        )

        # Process and print the response
        result = json.loads(response.get("body").read())
        input_tokens = result["usage"]["input_tokens"]
        output_tokens = result["usage"]["output_tokens"]
        output_list = result.get("content", [])

        print("Invocation details:")
        print(f"- The input length is {input_tokens} tokens.")
        print(f"- The output length is {output_tokens} tokens.")

        print(f"- The model returned {len(output_list)} response(s):")
        for output in output_list:
            print(output["text"])

        return result

    except ClientError as err:
        logger.error(
            "Couldn't invoke Claude 3 Sonnet. Here's why: %s: %s",
            err.response["Error"]["Code"],
            err.response["Error"]["Message"],
        )
        raise

In [ ]:
def load_text(filepath: str) -> str:
    """Function to read filepath to text file in and return one string of the entire file"""
    with open(filepath, "r") as f:
        return f.read()

In [ ]:
transcript = load_text("rapid7_dw_transcript.txt")
PROMPT_TEMPLATE = load_text("DWRO_prompt_template.txt")

In [ ]:
from langchain.prompts.prompt import PromptTemplate

pt = PromptTemplate(input_variables=["transcript"], template=PROMPT_TEMPLATE)

full_prompt = pt.format(transcript=transcript)

In [ ]:
kazu = invoke_claude_3_with_text(full_prompt)